<a href="https://colab.research.google.com/github/mjh09/DS-Unit-2-Tree-Ensembles/blob/master/Random_Forest_kaggle_pumps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Category_encoders

In [0]:
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

In [0]:
path = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Tree-Ensembles/master/data/tanzania/'

train = pd.merge(pd.read_csv(path + 'train_features.csv'),
                 pd.read_csv(path + 'train_labels.csv'))

test = pd.read_csv(path + 'test_features.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')

train, val = train_test_split(train, train_size=0.80, test_size=0.20,
                             stratify=train['status_group'], random_state=42)


In [0]:
def wrangle(X):
  # " Wrangles train, validate, and test sets in the same way"
  X = X.copy()
  
  # Convert date_recorded to datetime
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
  
  # Extract components from date_recored, then drop the original column
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
  X = X.drop(columns='date_recorded')
  
  # Engineer feature: how many years from construction_year to date_recorded
  X['years'] = X['year_recorded'] - X['construction_year']
  
  # Drop recorded by (never varies) and id (always varies, random)
  X = X.drop(columns=['recorded_by', 'id'])
  
  # Drop duplicate columns
  duplicate_columns = ['quantity_group']
  X = X.drop(columns=duplicate_columns)
  
  # About 3% of the time, latitude has small values near zero,
  # outside Tanzania, so we'll treat these like null values
  X['latitude'] = X['latitude'].replace(-2e-08, np.nan)
  
  # When columns have zeros and shouldn't, they are like null values
  cols_with_zeros = ['construction_year', 'longitude', 'latitude', 'gps_height', 'population']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)
    
  return X

In [0]:
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# combine the lists
features = numeric_features + categorical_features

In [7]:
# Arrange data into X features matrix and y target vector
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

# Make pipeline
pipeline = make_pipeline(ce.OrdinalEncoder(),
                         SimpleImputer(strategy='median'),
                         RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))

# Fit on train, score on validation, predict on test
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))
y_pred = pipeline.predict(X_test)

# write submission to csv file
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('random-forest-model.csv', index=False)

Validation Accuracy 0.8106902356902357
